<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-01-14 13:45:30
-------------------
qualified stocks: 89
with latest results: 84
still star stocks: 44
-------------------
Initial Investment:  1.04 C
CY Investment:  1.17 C
Reserve:  0.00
Current:  1.14 C
-------------------
Today PnL: 1.17 L (1.04%)
Current PnL: -14.86 L (-12.69%)
CY Booked + Current PnL: -3.36 L (-2.87%)
-------------------
Total profit:  3.23 L
Total loss:  -18.09 L
-------------------
Total Booked + Current PnL: 9.75 L (9.38%)
Total Booked PnL: 24.61 L (23.68%)
Curr Year Booked PnL: 11.50 L (10.1%)
Prev Year Booked PnL: 13.11 L (12.62%)
Est FTT:  1.87 C
Est FTT PnL: 73.00 L (64.08%)
-------------------
Est LTT:  2.48 C
Est LTT PnL: 1.34 C (117.73%)
Deployed:  1.04 C
Current:  1.14 C
CAGR/XIRR %: 7.76%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
12  BANKBARODA   195.74   283.0   -29.89       M-LC  23.60   87599.0   
15   BAYERCROP  4346.57  6805.0    12.59       H-SC  10.11   83495.0   
55   NESTLEIND  2228.06  2755.0   -19.16       X-LC  90.45  108586.0   
80         TCS  4164.02  4998.0     4.29       X-LC  76.40  165192.0   
48  LALPATHLAB  2630.06  3545.0   -18.40       H-SC  70.79  123856.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  XIRR  \
12      10282.0  24186.0        1.87         13.30  27.61  44.58  19.0   
15      13950.0  25382.0        1.64         20.06  30.40  56.56  21.0   
55       -589.0  26408.0       -0.95         -0.54  24.32  23.65   NaN   
80       2795.0  29735.0       -1.29          1.72  18.00  20.03  12.0   
48       8133.0  32128.0       -1.71          7.03  25.94  34.79  19.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
12     0.43        0.77     2.50       XR      NTT       BANKS  
15     0.55        0.73     6.90     X40N      NTT   CHEMICALS  
55    -0.02        0.95     3.08     XY25      NTT        FMCG  
80     0.09        1.45    15.93      X40      BTT          IT  
48     0.25        1.09    40.80     X40N      NTT  HEALTHCARE

In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
65        SAIL   130.64   228.0    41.68       M-MC  20.79  181602.0   
11  BANDHANBNK   217.54   400.0   -48.08       H-SC  19.10  158626.0   
56        NHPC    82.08   115.0   171.76       M-MC  44.94  130431.0   
3          ACC  2579.05  3906.0   -22.91       M-MC  25.84  141255.0   
27  EASEMYTRIP    18.68    26.4   -13.60       M-SC  24.72  130859.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  XIRR  \
65     -43360.0  211022.0        5.37        -19.27  116.20  74.53   6.0   
11     -80015.0  280181.0        5.03        -33.53  176.63  83.87 -25.0   
56     -10582.0   67146.0        4.62         -7.50   51.48  40.11 -85.0   
3      -47016.0  143882.0        4.06        -24.97  101.86  51.45 -41.0   
27     -35542.0  104308.0        3.82        -21.36   79.71  41.33 -93.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
65    -0.21        1.59     5.37     XY24      BTT    STEEL  
11    -0.29        1.39     5.03     XY24      NTT    BANKS  
56    -0.16        1.15     8.97     XY24      NTT    POWER  
3     -0.33        1.24     4.06     XY24      BTT   CEMENT  
27    -0.34        1.15     3.82     XY24      NTT   TRAVEL

In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction     RSP   Current  \
22   COFFEEDAY    59.14    80.00  -129.93       L-SC  100.00   55219.0   
50        LTIM  5664.39  7262.39     9.16       X-LC   22.47  172610.0   
36  HINDUNILVR  2443.56  3513.00   -14.07       X-LC   95.51  149760.0   
14   BATAINDIA  1550.24  2096.00   -26.03       M-SC   64.04  110838.0   
23      COLPAL  2837.00  3808.06    10.38       X-MC   74.16  135102.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  XIRR  \
22     -58330.0  98378.0       -5.02        -51.37  178.16  35.27   NaN   
50       2678.0  45258.0       -4.59          1.58   26.22  28.21   0.0   
36      -4184.0  71555.0       -3.01         -2.72   47.78  43.77  -5.0   
14     -17832.0  63133.0       -2.10        -13.86   56.96  35.20  -4.0   
23      -6748.0  55297.0       -1.82         -4.76   40.93  34.23 -54.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
22    -0.59        0.48    27.14       XR      NTT    HOTELS  
50     0.06        1.52    26.80     X200      ATH        IT  
36    -0.06        1.31    10.23      X40      BTT      FMCG  
14    -0.28        0.97     4.36      X40      NTT  FOOTWEAR  
23    -0.12        1.19    12.10      X40      ATH      FMCG

In [12]:
# top 5 to monitor
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
51      LUXIND  1448.19  2442.00    25.71       M-SC   4.49  158707.0   
15   BAYERCROP  4346.57  6805.00    12.59       H-SC  10.11   83495.0   
75       SUNTV   536.47  1092.00    10.94       M-SC  42.70  119938.0   
50        LTIM  5664.39  7262.39     9.16       X-LC  22.47  172610.0   
54  METROPOLIS  1399.27  3102.00     8.39       X-SC  12.36  125062.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%  XIRR  \
51      21129.0  73291.0        2.41         15.36  46.18   68.62  22.0   
15      13950.0  25382.0        1.64         20.06  30.40   56.56  21.0   
75      21764.0  79903.0        1.62         22.17  66.62  103.55  20.0   
50       2678.0  45258.0       -4.59          1.58  26.22   28.21   0.0   
54      31311.0  82766.0       -0.86         33.40  66.18  121.69  33.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy       Category  
51     0.29        1.39    55.64     X40N      NTT       TEXTILES  
15     0.55        0.73     6.90     X40N      NTT      CHEMICALS  
75     0.27        1.05    16.08     X40N      BTT  ENTERTAINMENT  
50     0.06        1.52    26.80     X200      ATH             IT  
54     0.38        1.10    20.00       XR      NTT     HEALTHCARE

In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[~df_portfolio_features['Criteria'].isin(['XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
38         IEX   133.39   219.00   -10.05       H-SC  30.34  185497.0   
12  BANKBARODA   195.74   283.00   -29.89       M-LC  23.60   87599.0   
54  METROPOLIS  1399.27  3102.00     8.39       X-SC  12.36  125062.0   
30       GLAND  1463.94  3442.00    -0.99       H-SC  82.02   90693.0   
50        LTIM  5664.39  7262.39     9.16       X-LC  22.47  172610.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  XIRR  \
38      36234.0  59563.0        3.05         24.27   32.11   64.18  28.0   
12      10282.0  24186.0        1.87         13.30   27.61   44.58  19.0   
54      31311.0  82766.0       -0.86         33.40   66.18  121.69  33.0   
30      11640.0  95173.0       -0.93         14.72  104.94  135.12  29.0   
50       2678.0  45258.0       -4.59          1.58   26.22   28.21   0.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
38     0.61        1.63    25.76       XR      NTT        MISC  
12     0.43        0.77     2.50       XR      NTT       BANKS  
54     0.38        1.10    20.00       XR      NTT  HEALTHCARE  
30     0.12        0.80     4.59       XR      NTT      PHARMA  
50     0.06        1.52    26.80     X200      ATH          IT

In [14]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
29       GICRE   335.06   583.0     4.10       M-MC  65.17  180131.0   
41        INFY  1436.24  2275.0     8.09       X-LC  87.64  201245.0   
10  BAJFINANCE  7055.29  9150.0   -27.03       X-LC  98.88  228472.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%   XIRR  \
29      33710.0  74646.0        1.13         23.02  41.44  74.00  137.0   
41      51876.0  35359.0       -1.38         34.73  17.57  58.40   37.0   
10       9758.0  55176.0        3.06          4.46  24.15  29.69   12.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
29     0.45        1.58    38.33     XY24      BTT  INSURANCE  
41     1.47        1.77    40.43      X40      BTT         IT  
10     0.18        2.01    17.36      X40      BTT    FINANCE

In [15]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction     RSP   Current  \
22   COFFEEDAY    59.14    80.00  -129.93       L-SC  100.00   55219.0   
10  BAJFINANCE  7055.29  9150.00   -27.03       X-LC   98.88  228472.0   
9   BAJAJFINSV  1473.47  2392.00   -14.90       X-LC   97.75  133177.0   
17   BRITANNIA  4845.40  6446.05    -0.40       X-LC   96.63  107263.0   
36  HINDUNILVR  2443.56  3513.00   -14.07       X-LC   95.51  149760.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%   XIRR  \
22     -58330.0  98378.0       -5.02        -51.37  178.16  35.27    NaN   
10       9758.0  55176.0        3.06          4.46   24.15  29.69   12.0   
9       18246.0  53404.0        1.99         15.88   40.10  62.34    5.0   
17        664.0  34549.0       -1.04          0.62   32.21  33.03  239.0   
36      -4184.0  71555.0       -3.01         -2.72   47.78  43.77   -5.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
22    -0.59        0.48    27.14       XR      NTT   HOTELS  
10     0.18        2.01    17.36      X40      BTT  FINANCE  
9      0.34        1.17    15.65      X40      BTT  FINANCE  
17     0.02        0.94     5.80       AR      ATH     FMCG  
36    -0.06        1.31    10.23      X40      BTT     FMCG

In [16]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       19.57
1     25       40.88
2     50       70.89

In [17]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    40.26
X40     19.48
X40N    14.52
XR      14.32
AR       6.13
XY25     2.33
X200     1.52
SR       1.42
Name: CurrAlloc%, dtype: float64

In [18]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
M-SC    28.06
X-LC    21.85
H-SC    17.68
M-MC    10.81
X-SC     5.66
X-MC     4.15
M-LC     3.34
H-MC     2.80
L-SC     2.64
H-LC     1.38
Name: CurrAlloc%, dtype: float64

In [19]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
BANKS               14.20        -17.89   73.71
FINANCE             12.54        -14.41   62.58
IT                   9.00         -8.64   63.24
FMCG                 7.78         -7.06   46.17
MISC                 7.37         -8.00   48.03
PAINTS               5.92        -21.88   57.94
CHEMICALS            5.76        -10.32   79.93
HEALTHCARE           4.90          2.13   46.30
INSURANCE            4.31          1.54   51.58
AUTO                 3.33         -8.81   56.65
FOOTWEAR             3.00        -20.25   59.98
POWER                2.23        -11.82   56.54
TRAVEL               2.19        -23.57   74.02
ENTERTAINMENT        2.13         -7.86   83.66
TEXTILES             2.06          2.02   49.45
CERAMICS             1.98        -26.22   77.57
STEEL                1.59        -23.88  116.20
JEWELLERY            1.55        -63.80  115.60
ENGINEERING          1.43        -11.60   54.59
REFINERIES           1.38         -4.25   23.87
CEMENT               1.24        -33.28  101.86
ELECTRICAL           1.07         -5.06   44.17
DURABLES             0.88         -0.44   50.04
AC                   0.86        -29.72  101.32
PHARMA               0.80         12.83  104.94
HOTELS               0.48       -105.63  178.16

In [20]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      3071387.0     30
XR        1384367.0     18
X40N      1078235.0     17
X40        817990.0     12
AR         654818.0      7
SR         183516.0      2
XY25        64047.0      2
X200        45258.0      1

In [21]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
M-SC        2309697.0     28
H-SC        1743646.0     16
M-MC         960194.0     10
X-LC         825694.0     14
X-SC         466502.0      6
L-SC         297261.0      4
X-MC         207212.0      4
H-MC         149786.0      2
M-LC         140911.0      3
H-LC          78907.0      1

In [22]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
M-SC       XY24      1077653.0     11
M-MC       XY24       748774.0      7
H-SC       XY24       656010.0      4
X-LC       X40        620653.0      9
M-SC       X40N       506970.0      7
           XR         472995.0      6
H-SC       XR         401127.0      5
           X40N       388744.0      6
           AR         297765.0      1
X-SC       XR         187731.0      2
L-SC       XR         164096.0      2
H-MC       XY24       149786.0      2
M-MC       XR         134232.0      2
M-SC       AR         126847.0      2
X-SC       SR         121417.0      1
M-LC       XY24       116725.0      2
X-LC       AR          95736.0      2
X-MC       X40N        95071.0      2
X-SC       X40N        87450.0      2
H-LC       X40         78907.0      1
M-MC       AR          77188.0      1
L-SC       XY24        75883.0      1
X-SC       XY24        69904.0      1
X-LC       XY25        64047.0      2
M-SC       X40         63133.0      1
           SR          62099.0      1
L-SC       AR          57282.0      1
X-MC       XY24        56844.0      1
           X40         55297.0      1
X-LC       X200        45258.0      1
M-LC       XR          24186.0      1

In [23]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
